In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pprint
import json
import time
import random
import re

In [2]:
def getSearchQuery(str):
    return str.replace(" ",'+')

In [3]:
def getParsedRequest(url):
    uagent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"
    req = requests.get(url, headers = {'user-agent': uagent})
    return bs(req.content, 'lxml')

In [4]:
def getDataFromItem(item, site):
    title = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
    url = title['href']
    page = getParsedRequest(url)
    rawParams = page.find('script', {'data-name': 'HH/GoogleDfpService'})
    params = json.loads(rawParams['data-params'])
    minSalary = params['vac_salary_from']
    try:
        minSalary = int(maxSalary)
    except:
        minSalary = None
    maxSalary = params['vac_salary_to']
    try:
        maxSalary = int(maxSalary)
    except:
        maxSalary = None
    curSalary = params['vac_salary_cur']
    curSalary
    return { 'title': title.text, 'url': url, 'min': minSalary, 'max': maxSalary, 'cur': curSalary, 'site': site }

In [5]:
print('SEARCH FOR:')
inpName = input()
vacName = getSearchQuery(inpName)

SEARCH FOR:
Data Scientist


In [6]:
pdList = []

In [7]:
buttonNext = True
queryString = "https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=" + vacName
while buttonNext:
    parsed = getParsedRequest(queryString)
    vlist = parsed.find_all('div', {'class': 'vacancy-serp-item'})
    for item in vlist:
        time.sleep(random.randint(1, 2))
        vacData = getDataFromItem(item, 'hh.ru')
        pdList.append(vacData)
    buttonNext = parsed.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})
    if buttonNext:
        queryString = 'https://hh.ru' + buttonNext['href']
    time.sleep(random.randint(1, 2))

In [8]:
def getSJSearchQuery(str):
    return str.replace(" ",'%20')

In [9]:
def dataFromSJItem(item):
    url = item.find('a', {'class': 'icMQ_'})
    url = url['href']
    title = item.find('div', {'class': '_3mfro'}).text
    page = getParsedRequest('https://www.superjob.ru' + url)
    zp = page.find('span', {'class': '_2Wp8I'})
    minZp = None
    maxZp = None
    cur = 'RUR'
    if zp:
        zp = zp.text.replace('\xa0', '')
        arr = re.findall(r'\d+', zp)
        if len(arr) == 1:
            minZp = arr[0]
        if len(arr) == 2:
            minZp = min(arr)
            maxZp = max(arr)
    return { 'title': title, 'url': url, 'min': minZp, 'max': maxZp, 'cur': None, 'site': 'superjob.ru'}

In [10]:
vacName = getSJSearchQuery(inpName)
sjString = 'https://www.superjob.ru/vacancy/search/?geo%5Bc%5D%5B0%5D=1&keywords=' + vacName
buttonNext = True
while buttonNext:
    parsed = getParsedRequest(sjString)
    vlist = parsed.find_all('div', {'class': 'f-test-vacancy-item'}) 
    for item in vlist:
        time.sleep(random.randint(1, 2))
        tmp = dataFromSJItem(item)
        pdList.append(tmp)
    buttonNext = parsed.find('a', {'rel': 'next'})
    urlNext = None
    if buttonNext:
        urlNext = buttonNext['href']
        sjString = 'https://www.superjob.ru' + urlNext
    time.sleep(random.randint(1, 2))

In [12]:
data = pd.DataFrame(pdList)

In [15]:
data

,cur,max,min,site,title,url
0,,NaN,None,hh.ru,Data Analyst (Senior),https://hh.ru/vacancy/33864288?query=Data%20Sc...
1,,NaN,None,hh.ru,Senior Data Scientist (Команда знаний),https://hh.ru/vacancy/33206558?query=Data%20Sc...
2,,NaN,None,hh.ru,Senior Data Scientist (Ценообразование),https://hh.ru/vacancy/33245471?query=Data%20Sc...
3,RUR,NaN,None,hh.ru,Data Scientist,https://hh.ru/vacancy/35011897?query=Data%20Sc...
4,RUR,250000.0,None,hh.ru,NLP engineer / Data scientist NLP,https://hh.ru/vacancy/34811160?query=Data%20Sc...
5,,NaN,None,hh.ru,Data scientist,https://hh.ru/vacancy/33418971?query=Data%20Sc...
6,RUR,250000.0,None,hh.ru,Data Scientist,https://hh.ru/vacancy/34689339?query=Data%20Sc...
7,RUR,250000.0,None,hh.ru,Senior Data Scientist / Главный риск-аналитик ...,https://hh.ru/vacancy/34664001?query=Data%20Sc...
8,,NaN,None,hh.ru,Junior Data Scientist,https://hh.ru/vacancy/34886072?query=Data%20Sc...
9,,NaN,None,hh.ru,Data scientist,https://hh.ru/vacancy/33997503?query=Data%20Sc...


In [16]:
#data.to_excel('output.xlsx')